In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
 #bilinear. Supports bilinear, nearest, bicubic, area, lanczos3, lanczos5, gaussian, mitchellcubic.

In [2]:
class ResidualUnit_default(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.block_layers = [keras.layers.Conv2D(filters, kernel_size=(3,4), strides=strides, padding='same', use_bias=False),
                             keras.layers.BatchNormalization(),
                             self.activation,
                             keras.layers.Conv2D(filters, kernel_size=(3,4), strides=1, padding='same', use_bias=False),
                             keras.layers.BatchNormalization()]
        
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [keras.layers.Conv2D(filters, kernel_size=(1,1), strides=strides, padding='same', use_bias=False),
                                keras.layers.BatchNormalization()]

    def call(self, x):
        inputs = x
        
        for layer in self.block_layers:
            x = layer(x)
        
        for layer in self.skip_layers:
            inputs = layer(inputs)
        
        return self.activation(x + inputs)

In [3]:
class ResidualUnit_bottleneck(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.block_layers = [keras.layers.Conv2D(filters//4, kernel_size=(1,1), strides=1, padding='same', use_bias=False),
                             keras.layers.BatchNormalization(),
                             self.activation,
                             keras.layers.Conv2D(filters//4, kernel_size=(3,4), strides=strides, padding='same', use_bias=False),
                             keras.layers.BatchNormalization(),
                             self.activation,
                             keras.layers.Conv2D(filters, kernel_size=(1,1), strides=1, padding='same', use_bias=False),
                             keras.layers.BatchNormalization()]
        
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [keras.layers.Conv2D(filters, kernel_size=(1,1), strides=strides, padding='same', use_bias=False),
                                keras.layers.BatchNormalization()]

    def call(self, x):
        inputs = x
        
        for layer in self.block_layers:
            x = layer(x)
        
        for layer in self.skip_layers:
            inputs = layer(inputs)
        
        return self.activation(x + inputs)

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
  directory='/kaggle/input/breast-cancer-screening-dbt-2d/sets/Train',
  seed=42,
  image_size=(240, 320),
  batch_size=16,
  color_mode='grayscale',
  label_mode='categorical',
  shuffle=True)

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
  directory='/kaggle/input/breast-cancer-screening-dbt-2d/sets/Validation',
  seed=42,
  image_size=(240, 320),
  batch_size=16,
  color_mode='grayscale',
  label_mode='categorical',
  shuffle=True)

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
  directory='/kaggle/input/breast-cancer-screening-dbt-2d/sets/Test',
  seed=42,
  image_size=(240, 320),
  batch_size=16,
  color_mode='grayscale',
  label_mode='categorical',
  shuffle=True)

In [66]:
resnet18_default = keras.models.Sequential()

resnet18_default.add(keras.layers.Conv2D(filters=64, kernel_size=(7,9), strides=2, padding='same', use_bias=False, input_shape=(240, 320, 1)))
resnet18_default.add(keras.layers.BatchNormalization())
resnet18_default.add(keras.layers.Activation(keras.activations.relu))
resnet18_default.add(keras.layers.MaxPool2D(pool_size=(3,4), strides=2, padding='same'))

prev_filters = 64
for filters in [64]*2 + [128]*2 + [256]*2 + [512]*2:
    if filters == prev_filters :
        strides = 1
    else:
        strides = 2
        
    resnet18_default.add(ResidualUnit_default(filters, strides))
    prev_filters = filters

resnet18_default.add(keras.layers.GlobalAvgPool2D())
resnet18_default.add(keras.layers.Flatten())
resnet18_default.add(keras.layers.Dense(units=4, activation='softmax'))

resnet18_default.summary()

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_530 (Conv2D)         (None, 120, 160, 64)      4032      
                                                                 
 batch_normalization_530 (Ba  (None, 120, 160, 64)     256       
 tchNormalization)                                               
                                                                 
 activation_18 (Activation)  (None, 120, 160, 64)      0         
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 60, 80, 64)       0         
 g2D)                                                            
                                                                 
 residual_unit_default_51 (R  (None, 60, 80, 64)       98816     
 esidualUnit_default)                                            
                                                     

In [ ]:
opt = keras.optimizers.Adam(0.035)
resnet18_default.compile(opt, loss='categorical_crossentropy', metrics=['Accuracy', 'AUC', 'Precision'])
resnet18_results = resnet18_default.fit(train_ds, batch_size=16, epochs=10, validation_data=val_ds)

In [67]:
resnet34_default = keras.models.Sequential()

resnet34_default.add(keras.layers.Conv2D(filters=64, kernel_size=(7,9), strides=2, padding='same', use_bias=False, input_shape=(240, 320, 1)))
resnet34_default.add(keras.layers.BatchNormalization())
resnet34_default.add(keras.layers.Activation(keras.activations.relu))
resnet34_default.add(keras.layers.MaxPool2D(pool_size=(3,4), strides=2, padding='same'))

prev_filters = 64
for filters in [64]*3 + [128]*4 + [256]*6 + [512]*3:
    if filters == prev_filters :
        strides = 1
    else:
        strides = 2
        
    resnet34_default.add(ResidualUnit_default(filters, strides))
    prev_filters = filters

resnet34_default.add(keras.layers.GlobalAvgPool2D())
resnet34_default.add(keras.layers.Flatten())
resnet34_default.add(keras.layers.Dense(units=4, activation='softmax'))

resnet34_default.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_550 (Conv2D)         (None, 120, 160, 64)      4032      
                                                                 
 batch_normalization_550 (Ba  (None, 120, 160, 64)     256       
 tchNormalization)                                               
                                                                 
 activation_19 (Activation)  (None, 120, 160, 64)      0         
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 60, 80, 64)       0         
 g2D)                                                            
                                                                 
 residual_unit_default_59 (R  (None, 60, 80, 64)       98816     
 esidualUnit_default)                                            
                                                     

In [ ]:
opt = keras.optimizers.Adam(0.035)
resnet34_default.compile(opt, loss='categorical_crossentropy', metrics=['Accuracy', 'AUC', 'Precision'])
resnet34_results = resnet34_default.fit(train_ds, batch_size=16, epochs=10, validation_data=val_ds)

In [68]:
resnet50_bottleneck = keras.Sequential()

resnet50_bottleneck = keras.models.Sequential()
resnet50_bottleneck.add(keras.layers.Conv2D(filters=64, kernel_size=(7,9), strides=2, padding='same', use_bias=False, input_shape=(240, 320, 1)))
resnet50_bottleneck.add(keras.layers.BatchNormalization())
resnet50_bottleneck.add(keras.layers.Activation(keras.activations.relu))
resnet50_bottleneck.add(keras.layers.MaxPool2D(pool_size=(3,4), strides=2, padding='same'))

prev_filters = 64
for filters in [256]*3 + [512]*4 + [1024]*6 + [2048]*3:
    if filters == prev_filters :
        strides = 1
    else:
        strides = 2
        
    resnet50_bottleneck.add(ResidualUnit_bottleneck(filters, strides))
    prev_filters = filters

resnet50_bottleneck.add(keras.layers.GlobalAvgPool2D())
resnet50_bottleneck.add(keras.layers.Flatten())
resnet50_bottleneck.add(keras.layers.Dense(units=4, activation='softmax'))

resnet50_bottleneck.summary()

In [69]:
opt = keras.optimizers.Adam(0.035)
resnet50_bottleneck.compile(opt, loss='categorical_crossentropy', metrics=['Accuracy', 'AUC', 'Precision'])
resnet50_results = resnet50_bottleneck.fit(train_ds, batch_size=16, epochs=10, validation_data=val_ds)

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_586 (Conv2D)         (None, 120, 160, 64)      4032      
                                                                 
 batch_normalization_586 (Ba  (None, 120, 160, 64)     256       
 tchNormalization)                                               
                                                                 
 activation_20 (Activation)  (None, 120, 160, 64)      0         
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 60, 80, 64)       0         
 g2D)                                                            
                                                                 
 residual_unit_bottleneck_66  (None, 30, 40, 256)      88576     
  (ResidualUnit_bottleneck)                                      
                                                     

In [70]:
resnet101_bottleneck = keras.Sequential()

resnet101_bottleneck = keras.models.Sequential()
resnet101_bottleneck.add(keras.layers.Conv2D(filters=64, kernel_size=(7,9), strides=2, padding='same', use_bias=False, input_shape=(240, 320, 1)))
resnet101_bottleneck.add(keras.layers.BatchNormalization())
resnet101_bottleneck.add(keras.layers.Activation(keras.activations.relu))
resnet101_bottleneck.add(keras.layers.MaxPool2D(pool_size=(3,4), strides=2, padding='same'))

prev_filters = 64
for filters in [256]*3 + [512]*4 + [1024]*23 + [2048]*3:
    if filters == prev_filters :
        strides = 1
    else:
        strides = 2
        
    resnet101_bottleneck.add(ResidualUnit_bottleneck(filters, strides))
    prev_filters = filters

resnet101_bottleneck.add(keras.layers.GlobalAvgPool2D())
resnet101_bottleneck.add(keras.layers.Flatten())
resnet101_bottleneck.add(keras.layers.Dense(units=4, activation='softmax'))

resnet101_bottleneck.summary()

In [71]:
opt = keras.optimizers.Adam(0.035)
resnet101_bottleneck.compile(opt, loss='categorical_crossentropy', metrics=['Accuracy', 'AUC', 'Precision'])
resnet101_results = resnet101_bottleneck.fit(train_ds, batch_size=16, epochs=10, validation_data=val_ds)

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_639 (Conv2D)         (None, 120, 160, 64)      4032      
                                                                 
 batch_normalization_639 (Ba  (None, 120, 160, 64)     256       
 tchNormalization)                                               
                                                                 
 activation_21 (Activation)  (None, 120, 160, 64)      0         
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 60, 80, 64)       0         
 g2D)                                                            
                                                                 
 residual_unit_bottleneck_82  (None, 30, 40, 256)      88576     
  (ResidualUnit_bottleneck)                                      
                                                     

In [72]:
resnet152_bottleneck = keras.Sequential()

resnet152_bottleneck = keras.models.Sequential()
resnet152_bottleneck.add(keras.layers.Conv2D(filters=64, kernel_size=(7,9), strides=2, padding='same', use_bias=False, input_shape=(240, 320, 1)))
resnet152_bottleneck.add(keras.layers.BatchNormalization())
resnet152_bottleneck.add(keras.layers.Activation(keras.activations.relu))
resnet152_bottleneck.add(keras.layers.MaxPool2D(pool_size=(3,4), strides=2, padding='same'))

prev_filters = 64
for filters in [256]*3 + [512]*8 + [1024]*36 + [2048]*3:
    if filters == prev_filters :
        strides = 1
    else:
        strides = 2
        
    resnet152_bottleneck.add(ResidualUnit_bottleneck(filters, strides))
    prev_filters = filters

resnet152_bottleneck.add(keras.layers.GlobalAvgPool2D())
resnet152_bottleneck.add(keras.layers.Flatten())
resnet152_bottleneck.add(keras.layers.Dense(units=4, activation='softmax'))

resnet152_bottleneck.summary()

In [73]:
opt = keras.optimizers.Adam(0.035)
resnet152_bottleneck.compile(opt, loss='categorical_crossentropy', metrics=['Accuracy', 'AUC', 'Precision'])
resnet152_results = resnet152_bottleneck.fit(train_ds, batch_size=16, epochs=10, validation_data=val_ds)

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_743 (Conv2D)         (None, 120, 160, 64)      4032      
                                                                 
 batch_normalization_743 (Ba  (None, 120, 160, 64)     256       
 tchNormalization)                                               
                                                                 
 activation_22 (Activation)  (None, 120, 160, 64)      0         
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 60, 80, 64)       0         
 g2D)                                                            
                                                                 
 residual_unit_bottleneck_11  (None, 30, 40, 256)      88576     
 5 (ResidualUnit_bottleneck)                                     
                                                     